# One-shot games

## Overview

- This is a database of laboratory experiments that study how people play games like the [Prisoner's dilemma](https://en.wikipedia.org/wiki/Prisoner's_dilemma), [Chicken](http://economics.wikia.com/wiki/Chicken_game), and [Stag Hunt](https://en.wikipedia.org/wiki/Stag_hunt). We hope studying games from many experiments can help us understand how people make decisions during games and how they learn to play over time. See [Games](#games) for more information on the database itself, how to use it, and how to contribute.

- This is also an attempt to replicate and work on [machine learning](https://en.wikipedia.org/wiki/Machine_learning) (ML) models that predict how people make decisions during games. We hope developing ML models from the many experiments in our database can help us understand how to best model the types of strategies people develop when playing games. See [ML models](#mlmodels) for more information on models, how to use them, and how to contribute.

## Games <a name="games"/>

You may contribute to this [database](https://github.com/polomsca/one-shot-games/blob/master/gamesmxn.csv), by submitting any experiment with two-player, one-shot, normal-form games in this format:

paper | game | matrixrow | matrixcol | choicerow | choicecol | shape | symmetric | n 
--- | --- | --- | --- | --- | --- | --- | --- | ---
stahlwilson1994 | 1 | 40 10 70; 20 80 0; 30 100 60 | | 11 40; 0 40; 29 40 | | 3 3 | 1 | 

- **paper** : Paper authors and date published. Add 'et al' after the first author for papers with more than two authors. See [Papers](#papers) for more information.
- **game** : Game number if there are multiple games in one experiment. 
- **matrixrow** : Row player's payoff matrix. Separate elements with a space. Separate rows with a semicolon.
- **matrixcol** : May be blank if player results are pooled.
- **choicerow** : Row player's choice frequencies separated by semicolons. Fractions use a space instead of a slash. 
- **choicecol** : May be blank if player results are pooled.
- **shape** : Number of rows followed by number of columns. Separate values by a space.
- **symmetric** : '1' if symmetric, '0' otherwise.
- **n** : Number of subjects if **choicerow** is expressed in decimals or if number of subjects is otherwise unclear.

### Papers <a name="papers"/>

- Haruvy et al (2001) : Modeling and testing for heterogeneity in observed strategic behavior

- Haruvy and Stahl (2007) : Equilibrium selection and bounded rationality in symmetric normal-form games

- Rogers et al (2009) : Heterogeneous quantal response equilibrium and cognitive hierarchies

- Stahl and Haruvy (2008) : Level-n bounded rationality and dominated strategies in normal-form games <sup>[1](#myfootnote1)</sup>

- Stahl and Wilson (1994) : Experimental evidence on players' models of other players

- Stahl and Wilson (1995) : On players' models of other players, theory and experimental evidence

- Goeree and Holt (2001) : Ten little treasures of game theory and ten intuitive contradictions 
    - (only games from "A Matching Pennies Game" and "The Kreps Game")

- Rydval and Ortmann (2005) : Loss avoidance as selection principle, evidence from simple stag-hunt games

- Haruvy and Stahl (1998) : An empirical model of equilibrium selection in symmetric normal-form games

- Costa-Gomes et al (1998) : Cognition and behavior in normal-form games, an experimental study 
    - (does not include TS treatment)

#### Footnotes

[<a name="myfootnote1">1</a>] Appendix D, Game 10 is printed incorrectly as

12 | 63 | 22 
--- | --- | ---
0 | 0 | 38 
55 | 25 | 40 
35 | 35 | 43 

Game 10 should be read instead as

0 | 12 | 63 
--- | --- | ---
25 | 0 | 0
0 | 55 | 25 
100 | 35 | 35 

## ML models <a name="mlmodels"/>

The following program is my attempt to replicate results from: 

- Hartford et al (2016) : Deep learning for predicting human strategic behavior ('GameNet' model)

## Session

In [302]:
import tensorflow as tf
import numpy as np
import pandas as pd
import string as string
from __future__ import division
import time
import scipy as sp
from scipy import stats
sess = tf.InteractiveSession()

## Data

In [2]:
import_csv = pd.read_csv('gamesmxn.csv')

### Combine same games
Currently not enough games to afford this.
Also does not appear to work in Python 3.

```python
for i in range(1, import_csv.shape[0]):
    if pd.DataFrame(import_csv['matrixrow'][0:i] == import_csv['matrixrow'][i]).values.any():
        repeatmat = import_csv['matrixrow'][0:i][import_csv['matrixrow'][0:i] == import_csv['matrixrow'][i]].index[0]
        choicesum = np.matrix(import_csv['choicerow'][repeatmat]) + np.matrix(import_csv['choicerow'][i])
        choicesum = string.replace(str(choicesum),'[','')
        choicesum = string.replace(str(choicesum),']]','')
        choicesum = string.replace(str(choicesum),']\n',';')
        import_csv.set_value(repeatmat, 'choicerow', choicesum)
        import_csv.drop([i], inplace=True)
```

### Select games to use

Currently we are only using games of size 3x3. We remove 'stahlwilson1995' due to current uncertainty about some of the results.

In [151]:
tmp = np.empty((import_csv.shape[0]))
tmp[:] = np.NAN
for i in range(import_csv.shape[0]):
    if (import_csv['shape'][i] == '3 3' 
        and import_csv['symmetric'][i]
        and import_csv['paper'][i] != 'stahlwilson1995') == 1:
        tmp[i] = i
index = tmp[~np.isnan(tmp)]

In [152]:
inputs_row = np.zeros((index.shape[0], 3, 3, 2))
inputs_col = np.zeros((index.shape[0], 3, 3, 2))
target_row = np.zeros((index.shape[0], 3, 1))
for j in range(index.shape[0]):
    Ur = np.matrix(import_csv['matrixrow'][int(index[j])])
    Ur = (Ur-np.mean(Ur))/np.std(Ur)
    Uc = np.transpose(Ur)
    ar = np.matrix(import_csv['choicerow'][int(index[j])])
    if ar.shape[1] == 2:
        ar = ar[:,0]/ar[:,1]
    inputs_row[j, :, :, 0] = Ur
    inputs_row[j, :, :, 1] = Uc
    inputs_col[j, :, :, 0] = Uc
    inputs_col[j, :, :, 1] = Ur
    target_row[j] = ar

## Model

### Parameters

#### Weights and bias

In [153]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

#### Convolution and pooling

In [154]:
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

In [155]:
def rw_pool(x,c):
    x_max = tf.reduce_max(x, axis=2)
    x_til = tf.tile(x_max,[1,3,1])
    x_sha = tf.reshape(x_til,[-1,3,3,c])
    return tf.transpose(x_sha, perm=[0,2,1,3])

def cw_pool(x,c):
    x_max = tf.reduce_max(x, axis=1)
    x_til = tf.tile(x_max,[1,3,1])
    return tf.reshape(x_til,[-1,3,3,c])

### Input and target

In [156]:
x_row = tf.placeholder(tf.float32, shape=[None, 3, 3, 2])
x_col = tf.placeholder(tf.float32, shape=[None, 3, 3, 2])
y_ = tf.placeholder(tf.float32, shape=[None, 3, 1])

### Hidden layer 1 (row player)

Tensorflow version `0.12.1` uses `tf.concat(axis, values, name='concat')`, not `tf.concat(values, axis, name='concat')`

In [157]:
x_pool1 = tf.concat([x_row, rw_pool(x_row, 2), cw_pool(x_row, 2)], 3)

In [158]:
W_conv1 = weight_variable([1, 1, 6, 50])
b_conv1 = bias_variable([50])
h_conv1 = tf.nn.relu(conv2d(x_pool1, W_conv1) + b_conv1)

### Hidden layer 2 (row player)

Tensorflow version `0.12.1` uses `tf.concat(axis, values, name='concat')`, not `tf.concat(values, axis, name='concat')`

In [159]:
x_pool2 = tf.concat([h_conv1, rw_pool(h_conv1, 50), cw_pool(h_conv1, 50)], 3)

In [160]:
W_conv2 = weight_variable([1, 1, 150, 50])
b_conv2 = bias_variable([50])
h_conv2 = tf.nn.relu(conv2d(x_pool2, W_conv2) + b_conv2)

In [161]:
keep_prob = tf.placeholder(tf.float32)
h_conv2_drop = tf.nn.dropout(h_conv2, keep_prob)

<div style="text-align: right"> <h3> Hidden layer 1 (col player) </h3> </div>

Tensorflow version `0.12.1` uses `tf.concat(axis, values, name='concat')`, not `tf.concat(values, axis, name='concat')`

In [162]:
x_pool1_col = tf.concat([x_col, rw_pool(x_col, 2), cw_pool(x_col, 2)], 3)

In [163]:
W_conv1_col = weight_variable([1, 1, 6, 50])
b_conv1_col = bias_variable([50])
h_conv1_col = tf.nn.relu(conv2d(x_pool1_col, W_conv1_col) + b_conv1_col)

<div style="text-align: right"> <h3> Hidden layer 2 (col player) </h3> </div>

In [164]:
x_pool2_col = tf.concat([h_conv1_col, rw_pool(h_conv1_col, 50), cw_pool(h_conv1_col, 50)], 3)

In [165]:
W_conv2_col = weight_variable([1, 1, 150, 50])
b_conv2_col = bias_variable([50])
h_conv2_col = tf.nn.relu(conv2d(x_pool2_col, W_conv2_col) + b_conv2_col)

In [166]:
keep_prob_col = tf.placeholder(tf.float32)
h_conv2_col_drop = tf.nn.dropout(h_conv2_col, keep_prob_col)

<div style="text-align: right"> <h3> Action response layer 0 (col player) </h3> </div>

In [167]:
W_rwsm0 = weight_variable([1, 1, 50, 50])
h_rdsm1 = conv2d(h_conv2_col_drop, W_rwsm0)

In [168]:
ar_rwsm0_col = tf.reduce_sum(h_conv2_col_drop, axis=2)
ar_sfmx0_col = tf.nn.softmax(ar_rwsm0_col, dim=1)
ar_sfmx0_col = tf.slice(ar_sfmx0_col, [0, 0, 0], [-1, 3, 1])

### Action response layer 1 (row player response to col player)

In [169]:
W_rdsm1 = weight_variable([1, 1, 50, 50])
h_rdsm1 = conv2d(h_conv2_drop, W_rdsm1)

In [170]:
ar_rdsm1 = tf.reduce_sum(h_rdsm1, axis=3)
ar_dtpt1 = tf.matmul(ar_rdsm1, ar_sfmx0_col)

In [327]:
y = tf.nn.softmax(ar_dtpt1, dim=1)

### Cost function

In [341]:
beta = 0.01
regularizer = (tf.nn.l2_loss(W_conv1) 
               + tf.nn.l2_loss(W_conv2) 
               + tf.nn.l2_loss(W_conv1_col) 
               + tf.nn.l2_loss(W_conv2_col)
               + tf.nn.l2_loss(W_rwsm0) 
               + tf.nn.l2_loss(W_rdsm1))
#cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y)+beta*regularizer)

### Optimization

In [342]:
train_step = tf.train.AdamOptimizer(0.0002,0.9,0.999,1e-8).minimize(cross_entropy)

## Training and testing with 10 times 10-fold cross-validation

We calculate KL divergence in scipy due to uncertainty in the KL divergence calculation in Tensorflow.

In [343]:
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [345]:
timetime = time.time()
test_accuracy_all = np.zeros([10,10])
test_NLL_all = np.zeros([10,10])
test_RMSE_all = np.zeros([10,10])
test_KLadhoc_all = np.zeros([10,10])
for k in range(1,11):    
    for j in range(1,11):
        sess.run(tf.global_variables_initializer())
        shuffle = np.random.permutation(range(inputs_row.shape[0]))
        index_train = np.concatenate([shuffle[0:inputs_row.shape[0]//10*(j-1)], 
                              shuffle[inputs_row.shape[0]//10*j:inputs_row.shape[0]]]) 
        index_tests = shuffle[inputs_row.shape[0]//10*(j-1):inputs_row.shape[0]//10*j]
        inputs_row_train = inputs_row[index_train]
        inputs_col_train = inputs_col[index_train]
        target_row_train = target_row[index_train]
        inputs_row_tests = inputs_row[index_tests]
        inputs_col_tests = inputs_col[index_tests]
        target_row_tests = target_row[index_tests]
        for i in range(1501):
            if i%5 == 0:
                perm_train = np.random.permutation(range(inputs_row_train.shape[0]))
            inputs_row_train_batch = inputs_row_train[perm_train[perm_train.shape[0]//5*(i%5):perm_train.shape[0]//5*(i%5+1)]]
            inputs_col_train_batch = inputs_col_train[perm_train[perm_train.shape[0]//5*(i%5):perm_train.shape[0]//5*(i%5+1)]]
            target_row_train_batch = target_row_train[perm_train[perm_train.shape[0]//5*(i%5):perm_train.shape[0]//5*(i%5+1)]]
            if i%1500 == 0:
                train_accuracy = accuracy.eval(feed_dict={
                        x_row: inputs_row_train_batch,
                        x_col: inputs_col_train_batch,
                        y_: target_row_train_batch, 
                        keep_prob: 1.0,
                        keep_prob_col: 1.0})
                train_NLL = cross_entropy.eval(feed_dict={
                        x_row: inputs_row_train_batch, 
                        x_col: inputs_col_train_batch,
                        y_: target_row_train_batch,
                        keep_prob: 1.0,
                        keep_prob_col: 1.0})
                print("step %d, train accuracy %g, train NLL %g"%(i, train_accuracy, train_NLL))
            train_step.run(feed_dict={x_row: inputs_row_train_batch, 
                                      x_col: inputs_col_train_batch, 
                                      y_: target_row_train_batch, 
                                      keep_prob: 0.8,
                                      keep_prob_col: 0.8})
        test_accuracy = accuracy.eval(feed_dict={
            x_row: inputs_row_tests,
            x_col: inputs_col_tests,
            y_: target_row_tests, 
            keep_prob: 1.0,
            keep_prob_col: 1.0})
        test_NLL = cross_entropy.eval(feed_dict={
            x_row: inputs_row_tests,
            x_col: inputs_col_tests,
            y_: target_row_tests, 
            keep_prob: 1.0,
            keep_prob_col: 1.0})
        print("test accuracy %g, test NLL %g"%(test_accuracy, test_NLL))
        test_accuracy_all[j-1][k-1] = test_accuracy
        test_NLL_all[j-1][k-1] = test_NLL
        compare_y = y.eval(feed_dict={
                            x_row: inputs_row_tests, 
                            x_col: inputs_col_tests, 
                            y_: target_row_tests, 
                            keep_prob: 1.0, 
                            keep_prob_col: 1.0})
        compare_y = compare_y.reshape(compare_y.shape[0], compare_y.shape[1])
        compare_y_ = y_.eval(feed_dict={
                                x_row: inputs_row_tests,
                                x_col: inputs_col_tests,
                                y_: target_row_tests, 
                                keep_prob: 1.0,
                                keep_prob_col: 1.0})
        compare_y_ = compare_y_.reshape(compare_y_.shape[0], compare_y_.shape[1])
        compare = pd.DataFrame(np.concatenate((compare_y, compare_y_), axis=1))
        compare.columns = ["y1","y2","y3","y_1","y_2","y_3"]
        compare['RMSE 1'] = (compare['y1'] - compare['y_1'])**2
        compare['RMSE 2'] = (compare['y2'] - compare['y_2'])**2
        compare['RMSE 3'] = (compare['y3'] - compare['y_3'])**2
        RMSE = np.sqrt((compare['RMSE 1'] + compare['RMSE 2'] + compare['RMSE 3']).sum()/(compare['RMSE 1'].shape[0]*3))
        print("RMSE %g"%(RMSE))
        test_RMSE_all[j-1][k-1] = RMSE
        KL_adhoc_tmp = np.zeros(compare.shape[0])
        for l in range(compare.shape[0]):
            KLy_y = sp.stats.entropy([compare.loc[l]['y_1'], compare.loc[l]['y_2'], compare.loc[l]['y_3']],
                            [compare.loc[l]['y1'], compare.loc[l]['y2'], compare.loc[l]['y3']])
            KLuniformy = sp.stats.entropy([1/3, 1/3, 1/3],
                            [compare.loc[l]['y1'], compare.loc[l]['y2'], compare.loc[l]['y3']])
            KL_adhoc_tmp[l] = KLy_y/KLuniformy
        KLadhoc = np.mean(KL_adhoc_tmp)
        print("KL %g"%(KLadhoc))
        test_KLadhoc_all[j-1][k-1] = KLadhoc
elapsedtime = time.time() - timetime

step 0, train accuracy 0.352941, train NLL 0.805027
step 1500, train accuracy 0.941176, train NLL 0.0242934
test accuracy 0.888889, test NLL 0.0242812
RMSE 0.269147
KL 0.399837
step 0, train accuracy 0.176471, train NLL 0.787561
step 1500, train accuracy 0.941176, train NLL 0.0208457
test accuracy 0.666667, test NLL 0.0208374
RMSE 0.319826
KL 0.692673
step 0, train accuracy 0.411765, train NLL 0.796069
step 1500, train accuracy 0.941176, train NLL 0.0219251
test accuracy 0.777778, test NLL 0.0219114
RMSE 0.260479
KL 0.431069
step 0, train accuracy 0.588235, train NLL 0.802569
step 1500, train accuracy 0.882353, train NLL 0.0222138
test accuracy 0.888889, test NLL 0.0221944
RMSE 0.239906
KL 0.400493
step 0, train accuracy 0.647059, train NLL 0.795531
step 1500, train accuracy 0.941176, train NLL 0.02211
test accuracy 0.777778, test NLL 0.0220988
RMSE 0.241543
KL 0.419953
step 0, train accuracy 0.588235, train NLL 0.792025
step 1500, train accuracy 1, train NLL 0.0220514
test accuracy 0.

In [347]:
print("elapsed time %g min"%(elapsedtime/60))
print("test accuracy mean %g"%(np.mean(test_accuracy_all)))
print("test accuracy variance %g"%(np.var(test_accuracy_all)))
print("test NLL mean %g"%(np.mean(test_NLL_all)))
print("test NLL variance %g"%(np.var(test_NLL_all)))
print("test RMSE mean %g"%(np.mean(test_RMSE_all)))
print("test RMSE variance %g"%(np.var(test_RMSE_all)))
print("test KL mean %g"%(np.mean(test_KLadhoc_all)))
print("test KL variance %g"%(np.var(test_KLadhoc_all)))

elapsed time 22.2861 min
test accuracy mean 0.883333
test accuracy variance 0.0119444
test NLL mean 0.0220452
test NLL variance 1.75304e-06
test RMSE mean 0.219372
test RMSE variance 0.00224954
test KL mean 0.357586
test KL variance 0.201006
